# Tải mô hình

In [3]:
import torch
import transformers

print(torch.__version__)
print(transformers.__version__)
print("CUDA:", torch.cuda.is_available())
print(torch.cuda.get_device_name(0))


2.1.2+cu118
4.43.4
CUDA: True
NVIDIA GeForce GTX 1650


d:\AGU\KhoaLuan\E-Egent\lab\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load biến môi trường

In [4]:
import os
from dotenv import load_dotenv

load_dotenv()

HF_TOKEN = os.getenv("HF_TOKEN")
assert HF_TOKEN is not None, "HF_TOKEN chưa được set trong .env"

print("HF token loaded")

HF token loaded


## Login Hugging Face (programmatic)

In [5]:
from huggingface_hub import login

login(token=HF_TOKEN)

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


## Tải mô hình về cục bộ

In [9]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

MODEL_PATH = "../models/Llama-3.1-8B/"

tokenizer = AutoTokenizer.from_pretrained(
    MODEL_PATH,
    use_fast=True   # hoặc bỏ hẳn tham số này
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH,
    torch_dtype=torch.float16,
    device_map="auto"
)

print("Model loaded OK")

Loading checkpoint shards: 100%|██████████| 4/4 [00:28<00:00,  7.22s/it]


Model loaded OK


In [15]:
from transformers import TextStreamer

prompt = "AI Agent là gì?"

streamer = TextStreamer(
    tokenizer,
    skip_prompt=True,
    skip_special_tokens=True
)

inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

model.generate(
    **inputs,
    max_new_tokens=200,
    temperature=0.7,
    do_sample=True,
    streamer=streamer
)

# print(tokenizer.decode(out[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


 AI Agent là gì?
AI Agent là gì? AI Agent là gì? Trong thế giới trí tuệ nhân tạo, thuật ngữ “AI Agent” có ý nghĩa quan trọng trong việc mô tả và phân loại các hệ thống máy tính có khả năng thực hiện một số tác vụ hoặc nhiệm vụ được chỉ định. AI Agent là viết tắt của “Artificial Intelligence Agent”, có nghĩa là “Tác nhân trí tuệ nhân tạo”.
Trong thế giới trí tuệ nhân tạo, thuật ngữ “AI Agent” có ý nghĩa quan trọng trong việc mô tả và phân loại các hệ thống máy tính có khả năng thực hiện một số tác 

KeyboardInterrupt: 

In [16]:
import torch

print("CUDA available:", torch.cuda.is_available())
print("GPU count:", torch.cuda.device_count())

if torch.cuda.is_available():
    print("GPU name:", torch.cuda.get_device_name(0))
    print("VRAM (GB):", torch.cuda.get_device_properties(0).total_memory / 1024**3)


CUDA available: True
GPU count: 1
GPU name: NVIDIA GeForce GTX 1650
VRAM (GB): 3.99969482421875


In [20]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch

MODEL_ID = "Qwen/Qwen2.5-0.5B-Instruct"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    quantization_config=bnb_config,
    device_map={"": "cuda"},
    low_cpu_mem_usage=True
)

model.eval()


OutOfMemoryError: CUDA out of memory. Tried to allocate 260.00 MiB. GPU 0 has a total capacty of 4.00 GiB of which 0 bytes is free. Of the allocated memory 3.27 GiB is allocated by PyTorch, and 172.73 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [16]:
prompt = "Việt Nam ở đâu?"

messages = [
    {"role": "system", "content": "You are Qwen, created by Alibaba Cloud. You are a helpful assistant."},
    {"role": "user", "content": prompt}
]

text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)

inputs = tokenizer(text, return_tensors="pt").to("cuda")

outputs = model.generate(
    **inputs,
    max_new_tokens=120,
    do_sample=False,
    repetition_penalty=1.1,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id
)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

system
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.
user
Việt Nam ở đâu?
assistant
Vi!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
